In [ ]:
import torch
from torch import nn
from torch.nn import Module, Linear, ReLU, Sequential, ModuleList
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import KarateClub

In [ ]:
device: torch.device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
from torch_geometric.datasets import Planetoid, Reddit
from torch_geometric.transforms import NormalizeFeatures
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures()).to(device)
data = dataset[0]

In [ ]:
class GraphNN(Module):
    def __init__(self, input_size: int, hidden_layer_size: int, classes: int, dropout_p=.7):
       super().__init__()
       self.layer1 = GCNConv(input_size, hidden_layer_size)
       self.layer2 = GCNConv(hidden_layer_size, hidden_layer_size)
       self.layer3 = GCNConv(hidden_layer_size, classes) # At the end, node features will be a distribution over each class
       self.dropout = nn.Dropout(dropout_p)

    def forward(self, x, edge_index):
        x = self.layer1(x, edge_index)
        x = x.relu()
        x = self.dropout(x)
        x = self.layer2(x, edge_index)
        x = x.relu()
        x = self.dropout(x)
        x = self.layer3(x, edge_index)
        return x

In [ ]:
model = GraphNN(dataset.num_node_features, 256, dataset.num_classes).to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

for i in range(101):
    loss = train()
    print(f"Epoch {i} Loss {loss:.3f}")

In [ ]:
# Test
model.eval()
out = model(data.x, data.edge_index)
pred = out.argmax(dim=1)
test_correct = pred[data.test_mask] == data.y[data.test_mask]
accuracy = int(test_correct.sum()) / int(data.test_mask.sum())
print(accuracy)